In [1]:
pip install cassandra-driver

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from cassandra.cluster import Cluster

# Connect to the local Cassandra instance
cluster = Cluster(['127.0.0.1'])  # Assuming Cassandra is running on localhost
session = cluster.connect()



In [4]:
session.execute("CREATE KEYSPACE IF NOT EXISTS bigdata WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3}")
session.set_keyspace("bigdata")

In [5]:
# Execute your Cassandra commands here
query = "CREATE TABLE IF NOT EXISTS test (id UUID PRIMARY KEY, amt FLOAT)"
session.execute(query)



In [8]:
from cassandra.cluster import Cluster
import json
# JSON data to insert
json_data = {
    "id": "123e4567-e89b-12d3-a456-426614174000",
    "amt": 100.50
}

# Convert JSON data to a string
json_string = json.dumps(json_data)

# Prepare and execute the INSERT statement with JSON data
insert_query = "INSERT INTO test JSON %s"
session.execute(insert_query, [json_string])

# Close the connection
# session.shutdown()
# cluster.shutdown()


In [9]:
# Prepare and execute the SELECT statement
select_query = "SELECT * FROM test"
result_set = session.execute(select_query)

# Process the result set
for row in result_set:
    print(row)


Row(id=UUID('123e4567-e89b-12d3-a456-426614174000'), amt=100.5)


In [ ]:
import csv
from cassandra.cluster import Cluster
import json
cluster = Cluster()
session = cluster.connect()
session.set_keyspace("bigdata")
session.execute("CREATE TABLE IF NOT EXISTS training (txn_id varchar PRIMARY KEY,cc_no bigint ,txn_time varchar, year smallint, month smallint, day smallint, hour smallint, minute smallint,amount float, cc_provider int, merchant varchar,merchant_index smallint, country_name varchar, country_code varchar,country_index smallint, age int, marital_status varchar,marital_status_index smallint,gender varchar,gender_index smallint,loan varchar,loan_index smallint, status varchar,status_index smallint)")
with open('50_50.csv', 'r',encoding="ISO-8859-1") as csvfile:
        reader=csv.reader(csvfile)
        for row in reader:
        	#print(row)
        	#msg=json.loads(row)
        	#print(msg)
        	dic={
        	"txn_id":row[0],
        	"cc_no":row[1],
        	"txn_time":row[2],
        	"year":row[3],
        	"month":row[4],
        	"day":row[5],
        	"hour":row[6],
        	"minute":row[7],
        	"amount":row[8],
        	"cc_provider":row[9],
        	"merchant":row[10],
            "merchant_index":row[11],
        	"country_name":row[12],
        	"country_code":row[13],
            "country_index":row[14],
        	"age":row[15],
        	"marital_status":row[16],
            "marital_status_index":row[17],
        	"gender":row[18],
            "gender_index":row[19],
        	"loan":row[20],
            "loan_index":row[21],
        	"status":row[22],
            "status_index":row[23]
        	}            
        	r = json.dumps(dic)
        	#print(r)
        	strq=" INSERT INTO training JSON "+"'"+r+"'"
        	session.execute(strq)

In [20]:
from cassandra.cluster import Cluster

# # Connect to Cassandra
# cluster = Cluster(['127.0.0.1'])  # Change to your Cassandra host
# session = cluster.connect('bigdata')  # Change to your keyspace

# Define CQL query to create a table
create_table_query = """
    CREATE TABLE IF NOT EXISTS bank (
        age INT,
        job TEXT,
        marital TEXT,
        education TEXT,
        default_col TEXT,
        balance INT,
        housing TEXT,
        loan TEXT,
        contact TEXT,
        day INT,
        month TEXT,
        duration INT,
        campaign INT,
        pdays INT,
        previous INT,
        poutcome TEXT,
        deposit TEXT,
        ts TIMESTAMP,
        PRIMARY KEY (age)
    );
"""
drop_query = 'drop table bank'
# Execute the create table query
session.execute(create_table_query)

# # Close the connection
# session.shutdown()
# cluster.shutdown()


In [29]:
########### testing bank csv
import csv
import json
import datetime
import time
from cassandra.cluster import Cluster

def insert_data_from_csv(csv_file, keyspace, table):
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for i, row in enumerate(reader):
            if i >= 100:
                break
            # Convert row to JSON format
            row['ts'] = int(time.time() * 1000)
            json_row = json.dumps(row)
            # Define CQL query to insert JSON data into Cassandra
            insert_query = f"INSERT INTO {keyspace}.{table} JSON '{json_row}';"
            # Execute the insert query
            session.execute(insert_query)

# Specify CSV file path, keyspace, and table name
csv_file_path = 'bank.csv'
keyspace_name = 'bigdata'
table_name = 'bank'

# Call the function to insert data from CSV into Cassandra
insert_data_from_csv(csv_file_path, keyspace_name, table_name)

# # Close the connection
# session.shutdown()
# cluster.shutdown()


In [32]:
import pandas as pd

def insert_data(df, keyspace, table):
    for row in df.to_dict(orient='records'):
        json_row = json.dumps(row)
        # Define CQL query to insert JSON data into Cassandra
        insert_query = f"INSERT INTO {keyspace}.{table} JSON '{json_row}';"
        # Execute the insert query
        session.execute(insert_query)

# Specify CSV file path, keyspace, and table name
df = pd.read_csv('bank.csv')
keyspace_name = 'bigdata'
table_name = 'bank'

for i in range(20):
    random_rows = df.sample(n=20)
    random_rows['ts'] = int(time.time() * 1000)
    insert_data(random_rows, keyspace_name, table_name)
    time.sleep(5)
# # Close the connection
# session.shutdown()
# cluster.shutdown()

KeyboardInterrupt: 

In [ ]:
# Close the connection
session.shutdown()
cluster.shutdown()